In [1]:
import pandas as pd
import pickle

In [2]:
yezhou = pd.read_csv('./May6.csv')

In [3]:
def high_level_code(lst):
    return [code[:3] for code in lst.split(',')]

In [4]:
yezhou['HIGH_LVL_DIAG'] = yezhou['DIAG_CODES'].apply(high_level_code)

In [5]:
yezhou['HIGH_LVL_DIAG'].unique

<bound method Series.unique of 0                                               [V30, V50]
1                           [577, 276, 008, 574, 401, 250]
2        [197, 584, 348, 197, 789, 157, 560, V49, 276, ...
3                 [571, 456, 305, 287, 135, 572, 272, 493]
4                                     [V30, V50, V29, V05]
                               ...                        
18337                                      [V30, 785, V05]
18338    [530, 348, 518, 507, 482, 008, 785, 599, V10, ...
18339    [038, 507, 785, 518, 530, 349, 707, 482, 599, ...
18340    [396, 414, 398, 403, 585, 426, 427, 250, V58, ...
18341    [155, 576, 486, 197, 198, 198, 576, 790, 512, ...
Name: HIGH_LVL_DIAG, Length: 18342, dtype: object>

In [6]:
diag_codes = {}
for i, s in enumerate(yezhou['HIGH_LVL_DIAG']):
    for w in s:
        diag_codes[w] = diag_codes.get(w, 1) + 1
diag_codes = {k: v for k, v in sorted(diag_codes.items(), key=lambda item: item[1], reverse=True)}

In [8]:
print('Diag High Level: %d, Proc High Level: %d' %(len(diag_codes), len(proc_codes)))

Diag High Level: 880, Proc High Level: 0


In [9]:
icd_groups = {
    0: '001-139',
    1: '140-239',
    2: '240-279',
    3: '280-289',
    4: '290-319',
    5: '320-389',
    6: '390-459',
    7: '460-519',
    8: '520-579',
    9: '580-629',
    10:'630-679',
    11:'680-709',
    12:'710-739',
    13:'740-759',
    14:'760-779',
    15:'780-799',
    16:'800-999',
    17:'V01-V91',
    18:'E000-E999'
}

In [10]:
def convert_hi_lvl(org_lst):
    hi_lvl = []
    for c in org_lst:
        if c.startswith('E'):
            hi_lvl.append(18)
        elif c.startswith('V'):
            hi_lvl.append(17)
        else:
            c = int(c)
            if c >= 800 and c <= 999:
                hi_lvl.append(16)
            elif c >= 780 and c <= 799:
                hi_lvl.append(15)
            elif c >= 760 and c <= 779:
                hi_lvl.append(14)
            elif c >= 740 and c <= 759:
                hi_lvl.append(13)
            elif c >= 710 and c <= 739:
                hi_lvl.append(12)
            elif c >= 680 and c <= 709:
                hi_lvl.append(11)
            elif c >= 630 and c <= 679:
                hi_lvl.append(10)
            elif c >= 580 and c <= 629:
                hi_lvl.append(9)
            elif c >= 520 and c <= 579:
                hi_lvl.append(8)
            elif c >= 460 and c <= 519:
                hi_lvl.append(7)
            elif c >= 390 and c <= 459:
                hi_lvl.append(6)
            elif c >= 320 and c <= 389:
                hi_lvl.append(5)
            elif c >= 290 and c <= 319:
                hi_lvl.append(4)
            elif c >= 280 and c <= 289:
                hi_lvl.append(3)
            elif c >= 240 and c <= 279:
                hi_lvl.append(2)
            elif c >= 140 and c <= 239:
                hi_lvl.append(1)
            elif c >= 1 and c <= 139:
                hi_lvl.append(0)
    return hi_lvl

In [11]:
yezhou['CODED_HIGH_LVL_DIAG'] = yezhou['HIGH_LVL_DIAG'].apply(convert_hi_lvl)

In [12]:
yezhou = yezhou.drop(columns=['Unnamed: 0'])

In [16]:
output_code = yezhou[['HADM_ID','CODED_HIGH_LVL_DIAG']]

In [20]:
output_code.to_csv('HIGH_LVL_ICD.csv')

In [22]:
with open('high_lvl_icd.pickle','wb') as f:
    pickle.dump(icd_groups,f,pickle.HIGHEST_PROTOCOL)

In [15]:
yezhou

,HADM_ID,TEXT,CATEGORY,DIAG_CODES,PROC_CODES,ind,padded_tokens,CLEANED TEXT,CLEAN_WORDS,STEM_WORDS,CODED_TEXT,HIGH_LVL_DIAG,CODED_HIGH_LVL_DIAG
0,190979.0,Admission Note mom is very supportive.\nOther...,Nursing/other Nursing/other Nursing/other Nurs...,"V3000,V502",640.0,0.0,"['Admission', 'Note', 'mom', 'is', 'very', 'su...",Admission Note mom is very supportive. Otherwi...,"['admission', 'note', 'mom', 'supportive', 'ot...","['admiss', 'note', 'mom', 'support', 'otherwis...","[65, 13, 790, 89, 680, 2640, 1, 2640, 1, 230, ...","[V30, V50]","[17, 17]"
1,187350.0,Admission Date:\nDischarge Date:\nDate of Birt...,Discharge summary Radiology Radiology Radiolog...,"5770,27651,00845,57450,4019,25000","5185.0,5187.0,3893.0",1.0,"['Admission', 'Date:', '<PAD>', '<PAD>', '<PAD...",Admission Date: Discharge Date: Date of Birth:...,"['admission', 'date', 'discharge', 'date', 'da...","['admiss', 'date', 'discharg', 'date', 'date',...","[65, 120, 77, 120, 120, 967, 1096, 316, 281, 1...","[577, 276, 008, 574, 401, 250]","[8, 2, 0, 8, 6, 2]"
2,133981.0,Admission Date:\nDischarge Date:\nDate of Birt...,Discharge summary ECG ECG Radiology Radiology ...,"1976,5845,34830,1977,78951,1572,56089,V4986,27...",5491.0,3.0,"['Admission', 'Date:', '<PAD>', '<PAD>', '<PAD...",Admission Date: Discharge Date: Date of Birth:...,"['admission', 'date', 'discharge', 'date', 'da...","['admiss', 'date', 'discharg', 'date', 'date',...","[65, 120, 77, 120, 120, 967, 1096, 316, 1226, ...","[197, 584, 348, 197, 789, 157, 560, V49, 276, ...","[1, 9, 5, 1, 15, 1, 8, 17, 2, 6, 9, 15, 15, 17..."
3,147056.0,Admission Date:\nDischarge Date:\nDate of Birt...,Discharge summary Radiology Radiology Radiolog...,"5712,45620,30500,2875,135,5723,2720,49390","4233.0,9905.0,9904.0",4.0,"['Admission', 'Date:', '<PAD>', '<PAD>', '<PAD...",Admission Date: Discharge Date: Date of Birth:...,"['admission', 'date', 'discharge', 'date', 'da...","['admiss', 'date', 'discharg', 'date', 'date',...","[65, 120, 77, 120, 120, 967, 1096, 316, 1226, ...","[571, 456, 305, 287, 135, 572, 272, 493]","[8, 6, 4, 3, 0, 8, 2, 7]"
4,148310.0,Neonatology Patient is 3.41 kg term newborn se...,Nursing/other Nursing/other,"V3001,V502,V290,V053","640.0,9955.0",5.0,"['Neonatology', 'Patient', 'is', '3.41', 'kg',...",Neonatology Patient is 3.41 kg term newborn se...,"['neonatology', 'patient', 'kg', 'term', 'newb...","['neonatolog', 'patient', 'kg', 'term', 'newbo...","[1282, 6, 368, 1590, 2064, 312, 1413, 268, 438...","[V30, V50, V29, V05]","[17, 17, 17, 17]"
...,...,...,...,...,...,...,...,...,...,...,...,...,...
18337,153176.0,BABYGRAM (CHEST ONLY) ...,Radiology Nursing/other Nursing/other Nursing/...,"V3001,7852,V053","640.0,9955.0",19993.0,"['BABYGRAM', '(CHEST', 'ONLY)', 'Clip', '<PAD>...",BABYGRAM (CHEST ONLY) Clip Reason: r/o CHD Adm...,"['babygram', 'chest', 'clip', 'reason', 'ro', ...","['babygram', 'chest', 'clip', 'reason', 'ro', ...","[4024, 14, 110, 20, 395, 1, 52, 92, 2064, 10, ...","[V30, 785, V05]","[17, 15, 17]"
18338,131551.0,Admission Date:\nDischarge Date:\nDate of Birt...,Discharge summary ECG ECG ECG ECG ECG ECG ECG ...,"53021,34830,51881,5070,4821,00845,78550,5990,V...","4516.0,966.0,3897.0,9604.0,9671.0,3891.0",19994.0,"['Admission', 'Date:', '<PAD>', '<PAD>', '<PAD...",Admission Date: Discharge Date: Date of Birth:...,"['admission', 'date', 'discharge', 'date', 'da...","['admiss', 'date', 'discharg', 'date', 'date',...","[65, 120, 77, 120, 120, 967, 1096, 316, 1226, ...","[530, 348, 518, 507, 482, 008, 785, 599, V10, ...","[8, 5, 7, 7, 7, 0, 15, 9, 17, 6, 12, 17, 17, 6..."
18339,152254.0,Admission Date:\nDischarge Date:\nService: MED...,Discharge summary Echo ECG ECG ECG Radiology R...,"0389,5070,78552,51881,5307,34982,70724,48241,5...","9671.0,9604.0,3891.0,3893.0,3897.0,14.0,966.0",19995.0,"['Admission', 'Date:', '<PAD>', '<PAD>', '<PAD...",Admission Date: Discharge Date: Service: MEDIC...,"['admission', 'date', 'discharge', 'date', 'se...","['admiss', 'date', 'discharg', 'date', 's

In [14]:
yezhou.to_csv('May6.csv')